In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

# 去除超标值
X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =666)

In [5]:
import sys
sys.path.append("../../")
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

## scikit-learn中的线性回归

In [6]:
# 对比一下之前引入KNN的方式
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression # 导入了封装了类的模块sklearn.linear_model

lin_reg = LinearRegression() # 创建实例

In [7]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
lin_reg.coef_

array([ -1.18919477e-01,   3.63991462e-02,  -3.56494193e-02,
         5.66737830e-02,  -1.16195486e+01,   3.42022185e+00,
        -2.31470282e-02,  -1.19509560e+00,   2.59339091e-01,
        -1.40112724e-02,  -8.36521175e-01,   7.92283639e-03,
        -3.81966137e-01])

In [9]:
lin_reg.intercept_

34.161435496245851

In [10]:
lin_reg.score(X_test, y_test)

0.81298026026584858

## 对于KNN算法这个思路来说，也可以用来解决回归问题

### KNN Regressor

## 数据归一化

In [11]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
standardScaler.fit(X_train, y_train)
X_train = standardScaler.transform(X_train)
X_test = standardScaler.transform(X_test)

In [12]:
from sklearn.neighbors import KNeighborsRegressor

# lin_reg
knn_reg = KNeighborsRegressor() # 创建实例
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test) # 评价标准

0.84664511530389497

## 因为kNN算法涉及的超参数较多，所以必须评估超参数

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "weights": ["uniform"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1, 6)]
    },
    {
        "weights": ["uniform"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1, 6)]
    },
    {
        "weights": ["distance"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1, 6)]
    },
    {
        "weights": ["distance"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1, 6)]
    }
]

knn_reg = KNeighborsRegressor() # 创建分类器实例
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train) # 这个并没有使用X_test, y_test

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}, {'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [14]:
grid_search.best_params_ # 曼哈顿距离

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [15]:
grid_search.best_score_ # 按道理这个值是应该大于

0.79917999890996905

In [16]:
grid_search.best_estimator_.score(X_test, y_test) # 这个可以和，经过网格搜索后的值

0.88099665099417701

In [17]:
grid_search.best_estimator_.score(X_train, y_train)

1.0

In [18]:
knn_reg = KNeighborsRegressor(n_neighbors=5,p=1,weights='distance') # 创建实例
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test) # 评价标准，可以发现和 grid_search.best_estimator_.score(X_test, y_test) 是相等的

0.88099665099417701

In [19]:
# 如何求 y_predict
y_predict = knn_reg.predict(X_test)
# 爱人比有人爱更值得羡慕，见闻，为啥事物总是呈现双面成对地出现？
from playML.metrics import r2_score
r2_score(y_test, y_predict) # 见鬼了，怎么和上面 knn_red.score 是一样的？knn_reg.score(X_test, y_test)  是 0.84664511530389497

0.88099665099417712